In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import pandas as pd
from datetime import datetime
from termcolor import colored

# Timer.
startTime = datetime.now()

DATA_PATH = "E:/Acoustic/mel_data.json"

# Path to save model.
MODEL_SAVE = 'E:/Acoustic/model_1_CRNN.h5'

HISTORY_SAVE = "E:/Acoustic/history_1_CRNN.csv"
ACC_SAVE = "E:/Acoustic/models_acc_1_CRNN.json"


def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mel"]) 
    y = np.array(data["labels"])
    return X, y


def prepare_datasets(test_size, validation_size):
    X, y = load_data(DATA_PATH)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    X_train = X_train[..., np.newaxis]  # 4-dim array: (# samples, # time steps, # coefficients, 1)
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test


def build_model(input_shape):
    # Create model.
    model = keras.Sequential()

    # 1st convolutional layer.
    model.add(keras.layers.Conv2D(16, (5, 5), activation='relu', input_shape=input_shape))
        # 16 kernels, and 5x5 grid size of kernel.
    model.add(keras.layers.MaxPool2D((5, 5), strides=(2, 2), padding='same'))
        # Pooling size 5x5.
    model.add(keras.layers.BatchNormalization())
        # Batch Normalization allows model to be more accurate and computations are faster.

    # Resize for RNN part.
    resize_shape = model.output_shape[2] * model.output_shape[3]
    model.add(keras.layers.Reshape((model.output_shape[1], resize_shape)))

    # RNN layer.
    model.add(keras.layers.LSTM(32, input_shape=input_shape, return_sequences=True))

    # Flatten the output and feed into dense layer.
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(32, activation='relu'))
        # 32 neurons.
    model.add(keras.layers.Dropout(0.3))
        # Reduces chances of over fitting.

    # Output layer that uses softmax activation.
    model.add(keras.layers.Dense(2, activation='softmax'))
        # 2 neurons --> depends on how many categories we want to predict.

    return model


def predict(model, X, y):
    # Random prediction post-training.
    X = X[np.newaxis, ...]

    prediction = model.predict(X)

    # Extract index with max value.
    predicted_index = np.argmax(prediction, axis=1)
    print("Expected index: {}, Predicted index: {}".format(y, predicted_index))


if __name__ == "__main__":
    # Create train, validation and test sets.
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)  # (test size, val size)

    # Early stopping.
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

    # Checkpoint.
    checkpoint = keras.callbacks.ModelCheckpoint(MODEL_SAVE, monitor='val_loss',
                                                 mode='min', save_best_only=True, verbose=1)

    # Build the CRNN network.
    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    model = build_model(input_shape)

    # Compile the network.
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])

    model.summary()

    # Train the CRNN.
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=16, epochs=1000,
                        callbacks=[callback, checkpoint])

    # Save history.
    hist = pd.DataFrame(history.history)

    # Save to csv:
    hist_csv = HISTORY_SAVE
    with open(hist_csv, mode='w') as f:
        hist.to_csv(f)

    print(
        colored("CRNN model has been trained and its training history has been saved to {}.".format(hist_csv), "green"))

    # Evaluate the CNN on the test set.
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    print("Accuracy on test set is: {}".format(test_accuracy))

    # Timer output.
    time = datetime.now() - startTime
    print(time)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 40, 86, 16)        416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 20, 43, 16)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 20, 43, 16)       64        
 ormalization)                                                   
                                                                 
 reshape (Reshape)           (None, 20, 688)           0         
                                                                 
 lstm (LSTM)                 (None, 20, 32)            92288     
                                                                 
 flatten (Flatten)           (None, 640)               0

Epoch 21/1000
16/16 [==============================] - ETA: 0s - loss: 0.0017 - accuracy: 1.0000
Epoch 21: val_loss improved from 0.00073 to 0.00062, saving model to E:/Acoustic\model_1_CRNN.h5
16/16 [==============================] - 1s 66ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 6.2300e-04 - val_accuracy: 1.0000
Epoch 22/1000
16/16 [==============================] - ETA: 0s - loss: 0.0023 - accuracy: 1.0000
Epoch 22: val_loss improved from 0.00062 to 0.00060, saving model to E:/Acoustic\model_1_CRNN.h5
16/16 [==============================] - 1s 65ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 5.9546e-04 - val_accuracy: 1.0000
Epoch 23/1000
16/16 [==============================] - ETA: 0s - loss: 0.0025 - accuracy: 1.0000
Epoch 23: val_loss improved from 0.00060 to 0.00058, saving model to E:/Acoustic\model_1_CRNN.h5
16/16 [==============================] - 1s 65ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 5.7687e-04 - val_accuracy: 1.0000
Epoch 24/1000
15

16/16 [==============================] - 1s 59ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 4.2841e-05 - val_accuracy: 1.0000
Epoch 72/1000
15/16 [===========================>..] - ETA: 0s - loss: 1.1957e-04 - accuracy: 1.0000
Epoch 72: val_loss improved from 0.00002 to 0.00002, saving model to E:/Acoustic\model_1_CRNN.h5
16/16 [==============================] - 1s 65ms/step - loss: 1.1392e-04 - accuracy: 1.0000 - val_loss: 1.6844e-05 - val_accuracy: 1.0000
Epoch 73/1000
15/16 [===========================>..] - ETA: 0s - loss: 2.2676e-04 - accuracy: 1.0000
Epoch 73: val_loss improved from 0.00002 to 0.00002, saving model to E:/Acoustic\model_1_CRNN.h5
16/16 [==============================] - 1s 64ms/step - loss: 2.1520e-04 - accuracy: 1.0000 - val_loss: 1.6340e-05 - val_accuracy: 1.0000
Epoch 74/1000
16/16 [==============================] - ETA: 0s - loss: 1.7645e-04 - accuracy: 1.0000
Epoch 74: val_loss improved from 0.00002 to 0.00002, saving model to E:/Acoustic\model_1_CRNN

In [3]:
# Make prediction on a random sample.
X = X_test[55]
y = y_test[55]
predict(model, X, y)

    # Save model accuracies on test set (for weight calculations later on).
accuracy = {
    "model_acc": [],
    "model_loss": [],
    "total_train_time": [],
}

accuracy["model_acc"].append(test_accuracy)
accuracy["model_loss"].append(test_error)
accuracy["total_train_time"].append(str(time))

with open(ACC_SAVE, "w") as fp:
    json.dump(accuracy, fp, indent=4)

1/1 [==============================] - 1s 951ms/step
Expected index: 0, Predicted index: [0]


In [4]:
y

0

In [5]:
import os
import json
import librosa
import tensorflow as tf
import numpy as np
from termcolor import colored

# Read and save parameters.
DATASET_PATH = "E:/Acoustic_test"  # Path of folder with testing audios
SAVED_MODEL_PATH = "E:/Acoustic/model_1_CRNN.h5"  # Path of trained model
SAMPLE_RATE = 22050  # Sample rate in Hz.
DURATION = 1  # Length of audio files fed. Measured in seconds.
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

# Predictions (1 or 0)
JSON_PATH = "E:/Acoustic_test/predictions_CRNN.json"
# Performance scores (accuracy, precision, recall, f1 score)
JSON_PERFORMANCE = "E:/Acoustic_test/model_scores_CRNN.json"

# Prediction of fed audio
class _Class_Predict_Service:
    """Singleton class for keyword spotting inference with trained models.
    :param model: Trained model
    """
    # Mapping so drone = 1.
    model = None
    _mapping = [
        1,
        0
    ]
    _instance = None

    # Predict hard values (1 or 0).
    def predict(self, file_path):
        """
        :param file_path (str): Path to audio file to predict
        :return predicted_keyword (str): Keyword predicted by the model
        """

        # Extract mels from testing audio.
        log_mel = self.preprocess(file_path)

        # We need a 4-dim array to feed to the model for prediction: (# samples, # time steps, # coefficients, 1).
        log_mel = log_mel[np.newaxis, ..., np.newaxis]

        # Get the predicted label.
        predictions = self.model.predict(log_mel)
        predicted_index = np.argmax(predictions)
        predicted_class = self._mapping[predicted_index]
        return predicted_class

    # Outputs certainty values for soft voting (1-0).
    def preprocess(self, file_path, n_mels=90, n_fft=2048, hop_length=512, num_segments=1):
        """Extract MFCCs from audio file.
        :param file_path (str): Path of audio file
        :param n_mels (int): # of mels to extract
        :param n_fft (int): Interval we consider to apply STFT. Measured in # of samples
        :param hop_length (int): Sliding window for STFT. Measured in # of samples
        """

        num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)

        # Load audio file.
        signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

        # Process segments extracting mels and storing data.
        for s in range(num_segments):
            start_sample = num_samples_per_segment * s  # s=0 --> 0
            finish_sample = start_sample + num_samples_per_segment  # s=0 --> num_samples_per_segment

            # Extract mel specs.
            mel = librosa.feature.melspectrogram(y=signal[start_sample:finish_sample], sr=sr, n_mels=n_mels, n_fft=n_fft,
                                                 hop_length=hop_length)
            log_mel = librosa.power_to_db(mel)

        return log_mel.T


def Keyword_Spotting_Service():
    """Factory function for Keyword_Spotting_Service class.
    :return _Keyword_Spotting_Service._instance (_Keyword_Spotting_Service):
    """

    # Ensure an instance is created only the first time the factory function is called.
    if _Class_Predict_Service._instance is None:
        _Class_Predict_Service._instance = _Class_Predict_Service()
        _Class_Predict_Service.model = tf.keras.models.load_model(SAVED_MODEL_PATH)
    return _Class_Predict_Service._instance


# Saving results into a json file.
def save_mfcc(dataset_path, json_path):
    # Dictionary to store data.
    data = {
        "mapping": [],  # Maps different class labels --> background is mapped to 0.
        "names": [],
        "results": [],  # mels are the training input, labels are the target.
    }

    # Loop through all the classes.
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # Ensure that we're not at the root level.
        if dirpath is not dataset_path:

            # Save the semantic label.
            dirpath_components = dirpath.split("/")  # class/background => ["class", "background"]
            semantic_label = dirpath_components[-1]  # considering only the last value
            data["mapping"].append(semantic_label)
            print("\nProcessing {}".format(semantic_label))

            # Process files for a specific class.
            for f in filenames:
                file_path = os.path.join(dirpath, f)

                # Create 2 instances of the keyword spotting service.
                kss = Keyword_Spotting_Service()
                kss1 = Keyword_Spotting_Service()

                # Check that different instances of the keyword spotting service point back to the same object.
                assert kss is kss1

                # Classify unseen audio.
                keyword = kss.predict(file_path)

                # Store mel for segment if it has the expected length.
                data["names"].append(f)
                data["results"].append(keyword)
                print("{}".format(file_path))

    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)


# Calculating performance scores (accuracy, precision, recall, f-score).
def performance_calcs(performance_path):
    # Dictionary to store model performance results.
    performance = {
        "TP": [],
        "FN": [],
        "TN": [],
        "FP": [],
        "Accuracy": [],
        "Precision": [],
        "Recall": [],
        "F1 Score": [],
    }

    with open(JSON_PATH, "r") as fp:
        data = json.load(fp)

    # Convert lists to numpy arrays.
    y = np.array(data["results"])

    a = float(sum(y[0:int(len(y) / 2)]))
    b = float(sum(y[int(len(y) / 2):int(len(y))]))

    # Calculating TP, TN, FP, FN.
    TP = a
    FN = int(len(y) / 2) - a
    FP = b
    TN = int(len(y) / 2) - b

    # Performance result calcs.
    Accuracy = (TP + TN) / (TP + TN + FN + FP)
    Precision = TP / (TP + FP)
    Recall = TP / (TP + FN)
    F1 = (2 * Precision * Recall) / (Precision + Recall)

    performance["TP"].append(TP)
    performance["FN"].append(FN)
    performance["TN"].append(TN)
    performance["FP"].append(FP)
    performance["Accuracy"].append(Accuracy)
    performance["Precision"].append(Precision)
    performance["Recall"].append(Recall)
    performance["F1 Score"].append(F1)

    with open(performance_path, "w") as fp:
        json.dump(performance, fp, indent=4)


if __name__ == "__main__":
    save_mfcc(DATASET_PATH, JSON_PATH)
    performance_calcs(JSON_PERFORMANCE)

    print(
        colored("CRNN model performance scores have been saved to {}.".format(JSON_PERFORMANCE), "green"))


Processing Acoustic_test\UAV
1/1 [==============================] - 0s 307ms/step
E:/Acoustic_test\UAV\drone+traffic_garden_center_carpark_213.wav
1/1 [==============================] - 0s 35ms/step
E:/Acoustic_test\UAV\drone+traffic_garden_center_carpark_154.wav
1/1 [==============================] - 0s 24ms/step
E:/Acoustic_test\UAV\drone+traffic_garden_center_carpark_155.wav
1/1 [==============================] - 0s 25ms/step
E:/Acoustic_test\UAV\drone+traffic_garden_center_carpark_156.wav
1/1 [==============================] - 0s 26ms/step
E:/Acoustic_test\UAV\drone+traffic_garden_center_carpark_157.wav
1/1 [==============================] - 0s 22ms/step
E:/Acoustic_test\UAV\drone+traffic_garden_center_carpark_158.wav
1/1 [==============================] - 0s 24ms/step
E:/Acoustic_test\UAV\drone+traffic_garden_center_carpark_159.wav
1/1 [==============================] - 0s 34ms/step
E:/Acoustic_test\UAV\drone+traffic_garden_center_carpark_160.wav
1/1 [============================